<a href="https://colab.research.google.com/github/MariaZharova/test_rec_systems/blob/main/Compare_LightFM_and_Microsoft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install lightfm recommenders scrapbook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 4.3 MB/s 
     |████████████████████████████████| 335 kB 40.6 MB/s 
     |████████████████████████████████| 12.4 MB 37.7 MB/s 
     |████████████████████████████████| 355 kB 48.2 MB/s 
     |████████████████████████████████| 11.8 MB 26.5 MB/s 
     |████████████████████████████████| 197 kB 47.2 MB/s 
     |████████████████████████████████| 636 kB 45.0 MB/s 
     |████████████████████████████████| 4.4 MB 43.1 MB/s 
     |████████████████████████████████| 61 kB 6.9 MB/s 
     |████████████████████████████████| 389 kB 59.2 MB/s 
     |████████████████████████████████| 101 kB 11.1 MB/s 
     |████████████████████████████████| 6.6 MB 23.6 MB/s 
     |████████████████████████████████| 132 kB 49.0 MB/s 
     |████████████████████████████████| 106 kB 48.8 MB/s 
     |████████████████████████████████| 423 kB 7.5 MB/s 
  Created wheel for lightfm: file

In [3]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from scipy.sparse import csr_matrix

In [4]:
import re
import sys
import os
import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import pandas as pd 

from collections import defaultdict
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.datasets.split_utils import filter_k_core

# Transformer Based Models
from recommenders.models.sasrec.model import SASREC
from recommenders.models.sasrec.ssept import SSEPT

# Sampler for sequential prediction
from recommenders.models.sasrec.sampler import WarpSampler
from recommenders.models.sasrec.util import SASRecDataSet

# Evaluation
from recommenders.evaluation.python_evaluation import precision_at_k

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data = pd.read_csv('./drive/MyDrive/ML/clicks_cleared.csv')
print(data.shape)
data.drop_duplicates(subset=['uid', 'offer_id'], inplace=True)

print(data.shape)
data.head()

(2040489, 11)
(509897, 11)


,Unnamed: 0,timestamp,hit_id,uid,platform,event_name,screen,offer_id,ptn_dadd,id,geo
0,0,2022-06-29 01:59:29,e4ae2d7f9eab4cb7,87603527,android,OpenOfferScreen,FavoritesScreen,274235341,2022-06-29,274235341,2
6,6,2022-06-30 19:34:27,7215c2b0c24048fd,90893006,ios,OpenOfferScreen,SearchResultsList,274235341,2022-06-30,274235341,2
9,9,2022-06-28 14:42:26,2827276a10114d02,79135200,ios,OpenOfferScreen,MapScreen,274235341,2022-06-28,274235341,2
12,12,2022-06-30 19:09:20,ef2253baf489479c,46459357,android,OpenOfferScreen,MapScreen,274235341,2022-06-30,274235341,2
15,15,2022-06-29 16:13:44,a02d5a7593e54df7,6913635,ios,OpenOfferScreen,PersonalFeedScreen,274235341,2022-06-29,274235341,2


### Prepare data

In [7]:
# take users that have >5 clicks
while not (data['offer_id'].value_counts()[data['offer_id'].value_counts() <= 5].empty) or \
      not (data['uid'].value_counts()[data['uid'].value_counts() <= 5].empty):
    offer_ids = data['offer_id'].value_counts()[data['offer_id'].value_counts() > 5].index
    data = data[data['offer_id'].isin(offer_ids)]
    uids = data['uid'].value_counts()[data['uid'].value_counts() > 5].index
    data = data[data['uid'].isin(uids)]

In [8]:
print(data.shape)
data.head()

(309037, 11)


,Unnamed: 0,timestamp,hit_id,uid,platform,event_name,screen,offer_id,ptn_dadd,id,geo
0,0,2022-06-29 01:59:29,e4ae2d7f9eab4cb7,87603527,android,OpenOfferScreen,FavoritesScreen,274235341,2022-06-29,274235341,2
6,6,2022-06-30 19:34:27,7215c2b0c24048fd,90893006,ios,OpenOfferScreen,SearchResultsList,274235341,2022-06-30,274235341,2
9,9,2022-06-28 14:42:26,2827276a10114d02,79135200,ios,OpenOfferScreen,MapScreen,274235341,2022-06-28,274235341,2
12,12,2022-06-30 19:09:20,ef2253baf489479c,46459357,android,OpenOfferScreen,MapScreen,274235341,2022-06-30,274235341,2
15,15,2022-06-29 16:13:44,a02d5a7593e54df7,6913635,ios,OpenOfferScreen,PersonalFeedScreen,274235341,2022-06-29,274235341,2


In [9]:
# add action column
data['action'] = 1

# encode, start with 0
offer_encoder = {off: ind for ind, off in enumerate(data['offer_id'].unique())}
data['offer_id_enc'] = data['offer_id'].map(offer_encoder)
uid_encoder = {uid: ind for ind, uid in enumerate(data['uid'].unique())}
data['uid_enc'] = data['uid'].map(uid_encoder)

In [10]:
# sort by time
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.sort_values(by='timestamp', inplace=True)
print(data.shape)
data.head(10)

(309037, 14)


,Unnamed: 0,timestamp,hit_id,uid,platform,event_name,screen,offer_id,ptn_dadd,id,geo,action,offer_id_enc,uid_enc
834748,834748,2022-06-27 21:00:00,2241fefec0494315,398271fb-efcf-4875-bce8-69e550e89b36,android,OpenOfferScreen,MapScreen,272813805,2022-06-28,272813805,2,1,6864,3995
360760,360760,2022-06-27 21:00:00,f522e557db8b4679,75545312,ios,OpenOfferScreen,PersonalFeedScreen,275048289,2022-06-28,275048289,2,1,2597,4688
1313747,1313747,2022-06-27 21:00:01,8eb54de70ee64369,92650098,android,OpenOfferScreen,MapScreen,273499093,2022-06-28,273499093,2,1,12336,664
1271705,1271705,2022-06-27 21:00:03,b745ae3ebd004c1f,f9dbc181-9be0-4df8-a1ab-6c8877a8fde7,android,OpenOfferScreen,NaN,274694106,2022-06-28,274694106,2,1,11818,4938
1591987,1591987,2022-06-27 21:00:04,cd7cbcace3e24f47,48505768,ios,OpenOfferScreen,RecommendationsScreen,274830501,2022-06-28,274830501,2,1,15943,7435
461664,461664,2022-06-27 21:00:04,de0f716f51764173,92657713,android,OpenOfferScreen,MapScreen,274238981,2022-06-28,274238981,2,1,3434,6465
216664,216664,2022-06-27 21:00:05,f7707fa30b5d46af,178523e8-df7c-4495-95a6-1e820c8a2dd5,android,OpenOfferScreen,NaN,267118079,2022-06-28,267118079,2,1,1475,14645
1605664,1605664,2022-06-27 21:00:06,8eea69af192a429d,73224555,android,OpenOfferScreen,MapScreen,275099925,2022-06-28,275099925,2,1,16107,17218
483921,483921,2022-06-27 21:00:06,a19b623fb55a4cfa,6932077,android,OpenOfferScreen,Undefined,275183897,2022-06-28,275183897,2,1,3601,8626
825391,825391,2022-06-27 21:00:08,b604f45546494ff9,76863963,ios,OpenOfferScreen,MapScreen,189350588,2022-06-28,189350588,2,1,6774,6518


# LightFM

In [11]:
# create train and test sets
#users = set(data['uid_enc'].values)
data_train = pd.DataFrame({})
data_test = pd.DataFrame({})

users = list(data['uid_enc'].value_counts().index)
counter = 0
for user in users:
    counter +=1
    if len(data[data['uid_enc'] == user]) < 3:
        data_train = data_train.append(data[data['uid_enc'] == user][['uid_enc', 'offer_id_enc', 'timestamp']], ignore_index = True)
        #data_test[user] = []
        print(counter)
    else:
        data_train = data_train.append(data[data['uid_enc'] == user].iloc[:-2][['uid_enc', 'offer_id_enc', 'timestamp']], ignore_index = True)
        data_test = data_test.append(data[data['uid_enc'] == user].iloc[-2][['uid_enc', 'offer_id_enc', 'timestamp']], ignore_index = True)

In [12]:
data_train['uid_enc'] = data_train['uid_enc'].astype(int)
data_train['offer_id_enc'] = data_train['offer_id_enc'].astype(int)

data_test['uid_enc'] = data_test['uid_enc'].astype(int)
data_test['offer_id_enc'] = data_test['offer_id_enc'].astype(int)

print(data_train.shape, data_test.shape)
data_train.dropna(inplace=True)
data_test.dropna(inplace=True)
print(data_train.shape, data_test.shape)

display(data_train.head())
display(data_test.head())

(288603, 3) (20434, 3)
(288603, 3) (20434, 3)


,uid_enc,offer_id_enc,timestamp
0,166,1075,2022-06-29 08:12:26
1,166,4008,2022-06-29 08:19:08
2,166,4965,2022-06-29 08:19:41
3,166,2111,2022-06-29 10:58:19
4,166,2695,2022-06-29 11:00:47


,uid_enc,offer_id_enc,timestamp
0,166,1474,2022-06-30 11:51:37
1,3398,9218,2022-06-30 13:59:48
2,1706,1246,2022-06-30 17:00:19
3,353,7026,2022-06-30 14:01:30
4,69,12859,2022-06-30 14:52:43


In [13]:
# create action columns for making csr matrix
data_train['action'] = 1
data_test['action'] = 1

In [14]:
# create sparse matrices
csr_user_item_train = csr_matrix((data_train['action'], (data_train['uid_enc'], data_train['offer_id_enc'])))
csr_user_item_train[:10, :10].toarray()

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [15]:
csr_user_item_train.shape

(20434, 20577)

In [16]:
# make test and train datasets
train_users = set(data_train['uid_enc'].values)
train_items = set(data_train['offer_id_enc'].values)
#print(train_users)
print(len(train_users))
print(len(train_items))

data_test = data_test[(data_test['uid_enc'].isin(train_users)) & 
                      (data_test['offer_id_enc'].isin(train_items))]

# make test dict
test_dict = {}
for user_id in data_test['uid_enc'].drop_duplicates().values:
    test_dict[user_id] = list(data_test[data_test['uid_enc'] == user_id]['offer_id_enc'])

20434
20577


In [17]:
# instantiate and train the model
model = LightFM(loss='warp', 
                max_sampled=19, 
                no_components=150, 
                user_alpha=0.000012403932688782397,
                item_alpha=0.00007600815597574313, 
                learning_rate=0.03702643399600358)
model.fit(csr_user_item_train, 
          epochs=195, 
          num_threads=31, 
          verbose=True) # параметр, показывающий прогресс обучения

Epoch: 100%|██████████| 195/195 [09:49<00:00,  3.02s/it]


In [18]:
train_items = set(data_train['offer_id_enc'].values)
all_items = list(train_items)

def get_predictions(user_id, topk=100):
    preds = model.predict(user_id, all_items)
    inds = np.argsort(preds)[::-1]
    return inds[:topk]

preds = {}

for user_id in data_test['uid_enc'].drop_duplicates().values:
    pred_user = get_predictions(int(user_id), topk=10)
    preds[user_id] = pred_user.tolist()

### Evaluate LightFM

In [19]:
def calculate_hit(y_true, y_score):
    """
        INPUT:
        y_true - list of true test values for one user
        y_score - list of predictions values foro one user
        OUTPUT:
        ans - hit :)
    """
    #print(int(y_true[0]), y_score)
    if int(y_true[0]) in y_score:
        return 1
    else:
        return 0

In [20]:
hit_fin = 0
for ind, val in test_dict.items():
    #print(val, preds[ind])
    hit_fin += calculate_hit(val, preds[ind])
    
hit_fin /= len(test_dict)

In [21]:
print("ОТВЕТ LightFM:", hit_fin)

ОТВЕТ LightFM: 0.04698052265831457


In [ ]:
"""from sklearn.utils import column_or_1d
from sklearn.utils.multiclass import type_of_target

def precision_at_k(y_true, y_score, k):
    """
        #INPUT:
        #y_true - list of true test values for one user
        #y_score - list of predictions values foro one user
        #k - level of precision
        #OUTPUT:
        #ans - precision at k :)
    """
    ans = 0
    counter = 0
    for row in y_true:
        counter += 1
        if counter > k:
            return ans
        if row in y_score:
            ans += 1 / k
    return ans

prec_fin = 0
for ind, val in test_dict.items():
    #print(val,  preds[ind])
    prec_fin += precision_at_k(val, preds[ind], 5)
    
prec_fin /= len(test_dict)"""

In [ ]:
###prec_fin

# SASRec

In [5]:
# с этого момента можно запускать заново, здесь вся необходимая прелобработка

# read file and drop duplictates
data = pd.read_csv('./drive/MyDrive/ML/clicks_cleared.csv')
data.drop_duplicates(subset=['uid', 'offer_id'], inplace=True)

# take users that have >5 clicks
while not (data['offer_id'].value_counts()[data['offer_id'].value_counts() <= 5].empty) or \
      not (data['uid'].value_counts()[data['uid'].value_counts() <= 5].empty):
    offer_ids = data['offer_id'].value_counts()[data['offer_id'].value_counts() > 5].index
    data = data[data['offer_id'].isin(offer_ids)]
    uids = data['uid'].value_counts()[data['uid'].value_counts() > 5].index
    data = data[data['uid'].isin(uids)]

# encode, start with 1 - условие SASRec
offer_encoder = {off: ind for ind, off in enumerate(data['offer_id'].unique())}
data['offer_id_enc'] = data['offer_id'].map(offer_encoder) + 1
uid_encoder = {uid: ind for ind, uid in enumerate(data['uid'].unique())}
data['uid_enc'] = data['uid'].map(uid_encoder) + 1

# sort by time and user id
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.sort_values(by=['uid_enc', 'timestamp'], inplace=True)
print(data.shape)
data.head(5)

(309037, 13)


,Unnamed: 0,timestamp,hit_id,uid,platform,event_name,screen,offer_id,ptn_dadd,id,geo,offer_id_enc,uid_enc
338037,338037,2022-06-28 01:27:16,5185a915b14e4289,87603527,android,OpenOfferScreen,MapScreen,275141155,2022-06-28,275141155,2,2415,1
732232,732232,2022-06-28 01:28:20,d2f3b48c57f64bd8,87603527,android,OpenOfferScreen,MapScreen,271507998,2022-06-28,271507998,2,5843,1
732255,732255,2022-06-28 01:29:39,7d7703e8be18432b,87603527,android,OpenOfferScreen,MapScreen,274244215,2022-06-28,274244215,2,5844,1
1431702,1431702,2022-06-28 01:30:35,fcfe4f790c43403a,87603527,android,OpenOfferScreen,MapScreen,275218539,2022-06-28,275218539,2,13801,1
1491726,1491726,2022-06-28 06:57:47,b65c2f26c086419d,87603527,android,OpenOfferScreen,Undefined,274236033,2022-06-28,274236033,2,14576,1


In [6]:
# create .txt file for input to model
data[['offer_id_enc',	'uid_enc']].to_csv('out.txt', sep="\t", header=False, index=False)

In [7]:
# create special data format for SAS
dataS = SASRecDataSet(filename='out.txt', col_sep='\t')
# split into train, test and validation
dataS.split()

In [10]:
# model variables
num_epochs = 50
batch_size = 200
RANDOM_SEED = 100  # Set None for non-deterministic result

lr = 0.001             # learning rate
maxlen = 50            # maximum sequence length for each user
num_blocks = 2         # number of transformer blocks
hidden_units = 100     # number of units in the attention calculation
num_heads = 1          # number of attention heads
dropout_rate = 0.1     # dropout rate
l2_emb = 0.0           # L2 regularization coefficient
num_neg_test = 20577     # number of negative examples per positive example


In [11]:
# sample negative examples
sampler = WarpSampler(dataS.user_train, dataS.usernum, dataS.itemnum, batch_size=batch_size, maxlen=maxlen, n_workers=3)

In [12]:
model = SASREC(item_num=dataS.itemnum,
               seq_max_len=maxlen,
               num_blocks=num_blocks,
               embedding_dim=hidden_units,
               attention_dim=hidden_units,
               attention_num_heads=num_heads,
               dropout_rate=dropout_rate,
               conv_dims = [100, 100],
               l2_reg=l2_emb,
               num_neg_test=num_neg_test)

In [13]:
# ЗДЕСЬ ОТВЕТ ДЛЯ num_neg_test=20577
with Timer() as train_time:
    t_test = model.train(dataS, sampler, num_epochs=num_epochs, batch_size=batch_size, lr=lr, val_epoch=5) #4

Evaluating...



epoch: 5, time: 19.303152780000346, valid (NDCG@10: 0.01803966949073337, HR@10: 0.0318)
epoch: 5, time: 19.303152780000346,  test (NDCG@10: 0.006983828596699759, HR@10: 0.0138)


Evaluating...



epoch: 10, time: 31.77181639599985, valid (NDCG@10: 0.04915277145044888, HR@10: 0.0844)
epoch: 10, time: 31.77181639599985,  test (NDCG@10: 0.01879868347887841, HR@10: 0.0376)


Evaluating...


...

Evaluating...



epoch: 45, time: 115.39536363200205, valid (NDCG@10: 0.09782726465707198, HR@10: 0.1688)
epoch: 45, time: 115.39536363200205,  test (NDCG@10: 0.043744744276329926, HR@10: 0.0874)


Evaluating...



epoch: 50, time: 129.21047015500153, valid (NDCG@10: 0.09804967201692032, HR@10: 0.1698)
epoch: 50, time: 129.21047015500153,  test (NDCG@10: 0.04651057611648179, HR@10: 0.0948)



epoch: 50, test (NDCG@10: 0.04548322452488012, HR@10: 0.0924)


In [ ]:
import random
from tqdm import tqdm

def get_predictions(data):
    """
        Модифицированный метод evaluation класса SASREC,
        главный результат - получаем предсказания для всех пользователей из датасета
    """
    usernum = data.usernum # max № of user
    itemnum = data.itemnum # max № of item
    train = data.user_train
    valid = data.user_valid
    test = data.user_test

    pred_dict = {}
    all_inputs = {}
    
    # насэмплим рандомных 10000 пользователей (или меньше, если их разнообразие небольшое:))
    if usernum > 10000:
        users = random.sample(range(1, usernum + 1), 10000)
    else:
        users = range(1, usernum + 1)
    
    # для каждого пользователя делаем оценку
    for u in tqdm(users, ncols=70, leave=False, unit="b"):

        if len(train[u]) < 1 or len(test[u]) < 1: # если для текущего пользователя нет ничего в train или test => continue
            continue
        # для input_seq
        seq = np.zeros([model.seq_max_len], dtype=np.int32)
        idx = model.seq_max_len - 1
        seq[idx] = valid[u][0]
        idx -= 1
        for i in reversed(train[u]): 
            seq[idx] = i
            idx -= 1
            if idx == -1: # если нет в train и valid => break
                break
        # для candidate
        rated = set(train[u]) # то, что оценил пользователь, из train'a
        rated.add(0)
        item_idx = [test[u][0]] # первым в последовательность помещаем тык из теста
        for _ in range(model.num_neg_test): # размер последовательностей получается фиксированный, задаётся в параметрах модели при инициализации
                                            # https://github.com/microsoft/recommenders/blob/main/examples/00_quick_start/sasrec_amazon.ipynb
            t = np.random.randint(1, itemnum + 1)
            while t in rated:
                t = np.random.randint(1, itemnum + 1) # генерим рандомно (?)
            item_idx.append(t)

        inputs = {}
        inputs["user"] = np.expand_dims(np.array([u]), axis=-1) # просто номер пользователя
        inputs["input_seq"] = np.array([seq]) # входная последовательность что тыкнул пользователь - ИЗ TRAIN И VALID!
        inputs["candidate"] = np.array([item_idx]) # объявки, для которых будем вычислять логиты
        all_inputs[inputs["user"][0][0]] = inputs # словарик для всех inputs

        # добавляем каждого пользователя в словарик предиктов
        pred_dict[inputs["user"][0][0]] = model.predict(inputs)

    return pred_dict, all_inputs

In [ ]:
#pred_dict, inputs = get_predictions(dataS)

In [ ]:
"""# нужно связать значения логитов и кандидатов, выбрать самые вероятные значения и проверить, были ли они в тесте
# сразу расчёт precision@k
k = 5
fin_prec = 0
for key in pred_dict.keys():
  tmp = np.stack((pred_dict[key][0], inputs[key]['candidate'][0]), axis=-1)
  tmp = tmp[tmp[:, 0].argsort()] # сортировка по вероятностям (логитам)
  topk = tmp[-k:, 1].astype(int)
  # проверим, есть ли этот топk в тесте
  tmp_prec = 0
  for val in topk:
    if val in dataS.user_test[key]:
      tmp_prec += 1/k
  fin_prec += tmp_prec

fin_prec /= len(pred_dict.keys())
print("ОТВЕТ", fin_prec)"""

ОТВЕТ 0.19084692267230316
